In [1]:
import random
import write_data
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import models
import model
from sklearn.metrics import accuracy_score

/home/mishuhashi/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
def read_dict(filepath):
        with open(filepath, 'r') as f:
            temp = f.read().split('\n')
        result = {}
        for line in temp:
            if line:
                temp2 = line.split()
                result[temp2[0]] = np.array(list(float(i) for i in temp2[1:]))
        return result

vocab_dict = read_dict('glove.6B/glove.6B.50d.txt')

In [3]:
vocab_index = {}
embed = []

In [4]:
index = 0
for key in vocab_dict:
    vocab_index[key] = index
    embed.append(vocab_dict[key])
    index += 1

In [5]:
vocab_index[r'\unknow'] = index
embed.append(np.array([random.random() for c in range(len(vocab_dict['.']))]))

In [7]:
embed = np.array(embed)

In [9]:
class Compression(nn.Module):
    def __init__(self, embedding, hidden_dim, output_dim):
        super(Compression, self).__init__()
        self.embedding = nn.Embedding(embedding.shape[0], embedding.shape[1])
        self.embedding.weight.data = torch.tensor(embedding)
        self.embedding.weight.requires_grad = False
        self.lstm = models.LSTM(embedding.shape[1], hidden_dim, output_dim)

    def forward(self, x):
        temp = self.embedding(x)
        y = self.lstm(temp.float())
        return y

In [10]:
import torch.utils.data as data
import torch.optim as optim
from tqdm import tqdm

In [11]:
class data_loader(data.Dataset):
    def read_file(self, file_path):
        with open(file_path, 'r') as f:
            temp = f.read().split('\n')
        if not temp[-1]:
            temp.pop()
        return temp
    def word2vect(self, list_sentences):
        result = []
        for sentence in list_sentences:
            temp = sentence.split(' ')
            if not temp[-1]:
                temp.pop()
            count = 0
            for i in temp:
                if i not in vocab_dict:
                    temp[count] = r'\unknow'
                count += 1
            index = list(vocab_index[word] for word in temp)
            result.append(np.array(index))
        return result
    def __init__(self, type = 0, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.type = type    
        if self.type == 0:
            self.train_data = self.word2vect(self.read_file('train.ori'))
            temp1 = self.read_file('train.bin')
            self.train_labels = []
            for label in temp1:
                temp3 = list(label)
                self.train_labels.append([int(i) for i in temp3])
            count = 0
            for i in self.train_data:
                t1 = i.shape[0]
                t2 = len(self.train_labels[count])
                t3 = self.train_labels[count]
                if t1 > t2:
                    i = t2 - 1
                    t3.pop()
                    while i < t1 - 1:
                        t3.append(0)
                        i += 1
                    t3.append(1)
                    self.train_labels[count] = t3
                if t1 < t2:
                    i = t2
                    t3.pop()
                    while i > t1 - 1:
                        t3.pop()
                        i -= 1
                    t3.append(1)
                    self.train_labels[count] = t3
                count += 1
        else:
            if self.type == 1:
                self.test_data = self.word2vect(self.read_file('test.ori'))
                temp1 = self.read_file('test.bin')
                self.test_labels = []
                for label in temp1:
                    temp3 = list(label)
                    self.test_labels.append([int(i) for i in temp3])
                count = 0
                for i in self.test_data:
                    t1 = i.shape[0]
                    t2 = len(self.test_labels[count])
                    t3 = self.test_labels[count]
                    if t1 > t2:
                        i = t2 - 1
                        t3.pop()
                        while i < t1 - 1:
                            t3.append(0)
                            i += 1
                        t3.append(1)
                        self.test_labels[count] = t3
                    if t1 < t2:
                        i = t2 - 1
                        t3.pop()
                        while i > t1 - 1:
                            t3.pop()
                            i -= 1
                        t3.append(1)
                        self.test_labels[count] = t3
                    count += 1
                
            else:
                self.valid_data = self.word2vect(self.read_file('valid.ori'))
                temp1 = self.read_file('valid.bin')
                self.valid_labels = []
                for label in temp1:
                    temp3 = list(label)
                    self.valid_labels.append([int(i) for i in temp3])
                count = 0
                for i in self.valid_data:
                    t1 = i.shape[0]
                    t2 = len(self.valid_labels[count])
                    t3 = self.valid_labels[count]
                    if t1 > t2:
                        i = t2 - 1
                        t3.pop()
                        while i < t1 - 1:
                            t3.append(0)
                            i += 1
                        t3.append(1)
                        self.valid_labels[count] = t3
                    if t1 < t2:
                        i = t2 - 1
                        t3.pop()
                        while i > t1 - 1:
                            t3.pop()
                            i -= 1
                        t3.append(1)
                        self.valid_labels[count] = t3
                    count += 1
                
    def __getitem__(self, index):
        if self.type == 0:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            if self.type == 1:
                img, target = self.test_data[index], self.test_labels[index]
            else:
                img, target = self.valid_data[index], self.valid_labels[index]
        return img, target
    def __len__(self):
        if self.type == 0:
            return len(self.train_data)
        else:
            if self.type == 1:
                return len(self.test_data)
            else:
                return len(self.valid_data)

In [12]:
train_set = data_loader(0)

In [13]:
test_set = data_loader(1)
valid_set = data_loader(2)

In [14]:
trainloader = data.DataLoader(train_set, batch_size=1, shuffle=False, num_workers=0)
testloader = data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0)
validloader = data.DataLoader(valid_set, batch_size=1, shuffle=False, num_workers=0)

In [15]:
model = Compression(embed,30, 2)
epoch = 2

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [17]:
for e in range(epoch):
    # Train
    train_loss = 0.0
    for feature, label in tqdm(trainloader, desc="Training"):
        feature, label = Variable(torch.LongTensor(feature)), Variable(torch.tensor(label).squeeze())
        label = label.reshape(trainloader.batch_size, -1)
        y_hat = model(feature)
        loss = criterion(y_hat, label)
        train_loss += loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Test
    test_loss = 0.0
    test_acc = 0.0
    y = []
    outputs = []
    count = 0
    for feature, label in tqdm(testloader, desc="Testing"):
        temp = label
        #y += label
        feature, label = Variable(torch.LongTensor(feature)), Variable(torch.tensor(label).squeeze())
        label = label.reshape(testloader.batch_size, -1)
        y_hat = model(feature)
        loss = criterion(y_hat, label)
        train_loss += loss.data.item()
        test_loss += loss.data.item()
        y_hat = y_hat.reshape(y_hat.shape[0], y_hat.shape[2], y_hat.shape[1])
        y_hat = np.argmax(y_hat.data, axis=2).view(testloader.batch_size, -1)
        count1 = 0
        total = 0
        for i in label:
            count2 = 0
            for j in i:
                if y_hat[count1][count2] == j:
                    test_acc += 1
                count2 += 1
                total += 1
            count1 += 1
        count += total
        #outputs += y_hat.tolist()
    test_acc = test_acc / count
    #test_acc = accuracy_score(y, outputs)
    
    # Valid
    valid_loss = 0.0
    valid_acc = 0.0
    y = []
    outputs = []
    count = 0
    for feature, label in tqdm(validloader, desc="Valid"):
        temp = label
        #y += label
        feature, label = Variable(torch.LongTensor(feature)), Variable(torch.tensor(label).squeeze())
        label = label.reshape(validloader.batch_size, -1)
        y_hat = model(feature)
        loss = criterion(y_hat, label)
        train_loss += loss.data.item()
        valid_loss += loss.data.item()
        y_hat = y_hat.reshape(y_hat.shape[0], y_hat.shape[2], y_hat.shape[1])
        y_hat = np.argmax(y_hat.data, axis=2).view(testloader.batch_size, -1)
        count1 = 0
        total = 0
        for i in label:
            count2 = 0
            for j in i:
                if y_hat[count1][count2] == j:
                    valid_acc += 1
                count2 += 1
                total += 1
            count1 += 1
        count += total
        #outputs += y_hat.tolist()
    valid_acc = valid_acc / count
    print("Epoch: {}".format(e + 1))
    print('Train loss: {}'.format(train_loss))
    print('Test loss: {} | Test accuracy: {}'.format(test_loss, test_acc))
    print('Valid loss: {} | Valid accuracy: {}'.format(valid_loss, valid_acc))
    print('------------------------------------------------')

Training:   0%|          | 9/7858 [00:00<01:35, 82.00it/s]

Epoch: 1
Train loss: 6217.809907749295
Test loss: 635.7367730736732 | Test accuracy: 0.5952827974653837
Valid loss: 631.1216077804565 | Valid accuracy: 0.6022762706579358
------------------------------------------------


Valid: 100%|██████████| 978/978 [00:05<00:00, 169.32it/s]

Epoch: 2
Train loss: 5864.409089520574
Test loss: 602.0436209142208 | Test accuracy: 0.6025971994054604
Valid loss: 600.6418030560017 | Valid accuracy: 0.6138525101340817
------------------------------------------------
